### 读取背景音乐

In [1]:
import wave
import numpy as np

# music = wave.open(r'background.wav', mode='rb')
# music.getparams()
# bg_data = music.readframes(nframes=32191488)
# bg_data = np.frombuffer(bg_data, dtype=np.int16)#音频的格式通常都是16位；buffer缓冲器
# bg_data.shape

bg_data = np.zeros([44100*15*60], dtype=np.int16)

In [2]:
ding = wave.open(r'息肉音频.wav', mode='rb')
ding.getparams()

_wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=39710, comptype='NONE', compname='not compressed')

In [3]:
nframes = 39710
nchannels=1
ding_data = ding.readframes(nframes=nframes)
ding_data = np.frombuffer(ding_data, dtype=np.int16)
# ding_data = ding_data.reshape(nframes, 2)
ding_data.shape

(39710,)

### 读取视频息肉出现时间点

In [4]:
import sys
import os
import subprocess
sys.path.append(r'../../NLP/MedicalRecord')

from Lib.MRRecordUtil import load_sheet_arr_dict

In [5]:
# 计算视频秒数
video_fps = 18
def calc_video_len(video_name, base_dir):
    img_folder = os.path.join(os.path.join(base_dir, 'images_crop'), video_name)
    return len(os.listdir(img_folder)) // video_fps

# 合成视频音频
def gen_audio(bg_data, ding_data, video_secs, framerate, polp_time_arr):
#     bg_data = bg_data[:video_secs*framerate, :]
    print(video_secs, framerate, video_secs*framerate)
    bg_data_trunc = bg_data[:(video_secs*framerate)]
    bg_len = bg_data_trunc.shape[0]
    print('bg_len: %s' % bg_len)
    if ding_data is not None:
        ding_len = ding_data.shape[0]
        ding_sec = ding_len / framerate
#         print(ding_len)
        lasttime = -1
        for ptime in polp_time_arr:
            if (lasttime == -1 or ptime - lasttime > ding_sec) and ptime < video_secs:
                start_frame = int(ptime*framerate)
                end_frame = start_frame + ding_len
                end_frame = end_frame if bg_len > end_frame else bg_len
#                 print(ptime, start_frame, end_frame)
#                 bg_data[start_frame:end_frame, :] = ding_data[0:(end_frame-start_frame), :]
                bg_data_trunc[start_frame:end_frame] = ding_data[0:(end_frame-start_frame)]
                lasttime = ptime
            
    return bg_data_trunc

# 写音频
def write_audio(audio_data, secs, framerate=44100, nchannels=1, sampwidth=2):
    fp = wave.Wave_write('./generated.wav')
    #设置fp的参数
    fp.setframerate(framerate)
    fp.setnframes(secs*framerate)
    fp.setnchannels(nchannels)
    fp.setsampwidth(sampwidth)

    #写入目录
    fp.writeframes(audio_data.tobytes())
    fp.close()
    
# 读取excel记录视频数据
def get_video_data_from_sheet(sheet_data):
    video_data = []
    for r in sheet_data:
        r_data = {}
        r_data['视频名'] = '%d' % r['视频名']
        r_data['所选参数'] = r['所选参数']
        r_data['息肉出现时间点'] = [float(e) for e in r['息肉出现时间点'].split('\n') if e != '']
        video_data.append(r_data)
    return video_data

def add_audio_for_video_data(v_data, video_folder, base_dir):
    video_secs = calc_video_len(v_data['视频名'], base_dir)
    print('video seconds: %s' % video_secs)
    audo_data = gen_audio(bg_data, ding_data, video_secs=video_secs, framerate=44100, polp_time_arr=v_data['息肉出现时间点'])
    write_audio(audo_data, video_secs)
    
    video_path = os.path.join(video_folder, '%s_%s.avi' % (v_data['视频名'], v_data['所选参数']))
    trans_video_path = os.path.join(video_folder, '%s_%s_audio.mp4' % (v_data['视频名'], v_data['所选参数']))
    cmd = 'ffmpeg -i generated.wav -i %s %s' % (video_path, trans_video_path)
    subprocess.call(cmd)

In [7]:
excel = '息肉出现时间点统计.xls'
# r'D:\项目资料\息肉假阳性\202208\有息肉肠镜视频', 
for base_dir in [r'D:\项目资料\息肉假阳性\202208\无息肉肠镜视频']:
    video_folder = os.path.join(base_dir, 'videos_gen')
    sheet_data = load_sheet_arr_dict(os.path.join(base_dir, excel), 'Sheet1')
    video_data = get_video_data_from_sheet(sheet_data)
    
    for v_data in video_data:
        if len(v_data['息肉出现时间点']) > 0: # and v_data['视频名'] in ['3478', '3444']:
            print('processing ', v_data['视频名'])
            print('bg_data: %s, ding_data: %s' % (bg_data.shape[0], ding_data.shape[0]))
            add_audio_for_video_data(v_data, video_folder, base_dir)


processing  3419
bg_data: 39690000, ding_data: 39710
video seconds: 481
481 44100 21212100
bg_len: 21212100
60.889 2685204 2724914
62.5 2756250 2795960
111.5 4917150 4956860
127.778 5635009 5674719
300.333 13244685 13284395
456.056 20112069 20151779
465.167 20513864 20553574
475.333 20962185 21001895
processing  3434
bg_data: 39690000, ding_data: 39710
video seconds: 320
320 44100 14112000
bg_len: 14112000
6.278 276859 316569
12.889 568404 608114
30.167 1330364 1370074
210.611 9287945 9327655
236.333 10422285 10461995
240.944 10625630 10665340
242.167 10679564 10719274
299.833 13222635 13262345
processing  3438
bg_data: 39690000, ding_data: 39710
video seconds: 374
374 44100 16493400
bg_len: 16493400
18.167 801164 840874
43.167 1903664 1943374
104.722 4618240 4657950
106.0 4674600 4714310
108.389 4779954 4819664
109.389 4824054 4863764
162.056 7146669 7186379
168.722 7440640 7480350
175.5 7739550 7779260
176.722 7793440 7833150
308.5 13604850 13644560
347.833 15339435 15379145
349.556 

processing  3486
bg_data: 39690000, ding_data: 39710
video seconds: 340
340 44100 14994000
bg_len: 14994000
2.611 115145 154855
4.5 198450 238160
25.611 1129445 1169155
38.944 1717430 1757140
40.389 1781154 1820864
43.611 1923245 1962955
45.944 2026130 2065840
58.5 2579850 2619560
60.222 2655790 2695500
64.222 2832190 2871900
65.944 2908130 2947840
99.667 4395314 4435024
162.944 7185830 7225540
167.944 7406330 7446040
173.833 7666035 7705745
189.111 8339795 8379505
192.556 8491719 8531429
218.833 9650535 9690245
228.722 10086640 10126350
231.389 10204254 10243964
234.389 10336554 10376264
249.611 11007845 11047555
254.5 11223450 11263160
261.222 11519890 11559600
269.722 11894740 11934450
277.0 12215700 12255410
282.222 12445990 12485700
286.278 12624859 12664569
307.111 13543595 13583305
309.333 13641585 13681295
314.833 13884135 13923845
316.778 13969909 14009619
320.056 14114469 14154179
321.222 14165890 14205600
328.889 14504004 14543714
331.389 14614254 14653964
336.722 14849440 1

processing  3492
bg_data: 39690000, ding_data: 39710
video seconds: 401
401 44100 17684100
bg_len: 17684100
0.056 2469 42179
10.611 467945 507655
12.389 546354 586064
15.333 676185 715895
20.056 884469 924179
25.0 1102500 1142210
29.333 1293585 1333295
31.056 1369569 1409279
33.0 1455300 1495010
35.722 1575340 1615050
37.222 1641490 1681200
38.222 1685590 1725300
52.0 2293200 2332910
53.5 2359350 2399060
57.778 2548009 2587719
61.056 2692569 2732279
64.167 2829764 2869474
76.833 3388335 3428045
105.722 4662340 4702050
107.389 4735854 4775564
119.889 5287104 5326814
123.611 5451245 5490955
125.056 5514969 5554679
130.889 5772204 5811914
132.111 5826095 5865805
134.833 5946135 5985845
141.944 6259730 6299440
143.5 6328350 6368060
147.722 6514540 6554250
155.111 6840395 6880105
157.722 6955540 6995250
163.167 7195664 7235374
166.667 7350014 7389724
187.111 8251595 8291305
188.778 8325109 8364819
195.278 8611759 8651469
197.111 8692595 8732305
200.944 8861630 8901340
203.833 8989035 902874

processing  3466
bg_data: 39690000, ding_data: 39710
video seconds: 382
382 44100 16846200
bg_len: 16846200
0.056 2469 42179
2.389 105354 145064
3.5 154350 194060
5.222 230290 270000
8.0 352800 392510
9.833 433635 473345
11.611 512045 551755
17.833 786435 826145
22.556 994719 1034429
24.5 1080450 1120160
27.333 1205385 1245095
29.0 1278900 1318610
30.667 1352414 1392124
34.722 1531240 1570950
37.833 1668435 1708145
38.778 1710109 1749819
39.833 1756635 1796345
46.056 2031069 2070779
47.444 2092280 2131990
49.722 2192740 2232450
53.889 2376504 2416214
57.722 2545540 2585250
60.556 2670519 2710229
62.556 2758719 2798429
67.222 2964490 3004200
70.0 3087000 3126710
71.722 3162940 3202650
74.167 3270764 3310474
77.5 3417750 3457460
79.389 3501054 3540764
83.889 3699504 3739214
84.944 3746030 3785740
86.0 3792600 3832310
89.0 3924900 3964610
90.778 4003309 4043019
96.278 4245859 4285569
100.611 4436945 4476655
101.556 4478619 4518329
106.0 4674600 4714310
108.056 4765269 4804979
112.389 4956

processing  3475
bg_data: 39690000, ding_data: 39710
video seconds: 398
398 44100 17551800
bg_len: 17551800
0.333 14685 54395
1.722 75940 115650
3.389 149454 189164
6.056 267069 306779
8.889 392004 431714
10.0 441000 480710
11.167 492464 532174
13.556 597819 637529
15.056 663969 703679
22.5 992250 1031960
32.5 1433250 1472960
36.167 1594964 1634674
38.444 1695380 1735090
40.444 1783580 1823290
41.833 1844835 1884545
44.333 1955085 1994795
51.5 2271150 2310860
52.444 2312780 2352490
53.722 2369140 2408850
55.667 2454914 2494624
56.778 2503909 2543619
57.833 2550435 2590145
59.444 2621480 2661190
60.556 2670519 2710229
64.389 2839554 2879264
66.333 2925285 2964995
69.333 3057585 3097295
70.667 3116414 3156124
74.5 3285450 3325160
76.0 3351600 3391310
77.722 3427540 3467250
80.5 3550050 3589760
86.111 3797495 3837205
90.611 3995945 4035655
93.722 4133140 4172850
97.111 4282595 4322305
101.611 4481045 4520755
108.278 4775059 4814769
109.333 4821585 4861295
113.889 5022504 5062214
128.167 5

In [11]:
704 * 44100

31046400